위에 수정을 클릭 후 노트 설정에서 GPU - T4 설정을 해줍니다, 그 이후 Google mount를 진행합니다.


In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
# NVCC 플러그인 설치, C/C++을 사용하여 CUDA를 이용하기 때문에 필요합니다.

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-ehhsq_1w
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-ehhsq_1w
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=297775313e48ab44495295cd57a3dd91c7adefe7bee1b8e5a9d2878fd6d2d386
  Stored in directory: /tmp/pip-ephem-wheel-cache-tllbwz61/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin
# 이후 nvcc 플러그인 load

created output directory at /content/src
Out bin /content/result.out


In [ ]:
#test code입니다.
# %% 뒤에 파일 확장자를 써주면 됩니다.

%%cu
#include <stdio.h>

__global__ void hello(void){
    printf("hello CUDA %d!\n", threadIdx.x);
}

int main(void){
    hello<<<1,8>>>();
#if defined(__linux__)
  cudaDeviceSynchronize();
#endif
    fflush(stdout);
    return 0;
}

hello CUDA 0!
hello CUDA 1!
hello CUDA 2!
hello CUDA 3!
hello CUDA 4!
hello CUDA 5!
hello CUDA 6!
hello CUDA 7!



In [3]:
# 드라이브와 연결하는 과정입니다.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. SAXPY 기능 구현

In [4]:
# g드라이브에 올려진 asst3 폴더 주소를 입력하여 해당 폴더로 이동합니다.

%cd /saxpy

/content/drive/MyDrive/대학원 관련/c언어/asst3-master/asst3-master/saxpy


In [ ]:
!dir

cudaSaxpy  CycleTimer.h  main.cpp  Makefile  objs  saxpy.cu


In [5]:
!make

# 동작 확인

mkdir -p objs/
g++ -m64 -O3 -Wall -o cudaSaxpy objs/main.o  objs/saxpy.o -L/usr/local/cuda/lib64/ -lcudart


In [ ]:
!dir

# objs 폴더와 cudaSaxpy가 만들어짐

cudaSaxpy  CycleTimer.h  main.cpp  Makefile  objs  saxpy.cu


In [6]:
!./cudaSaxpy

# 전송 시간과 데이터를 옮기는 과정을 제외하고 시간을 측정합니다.

mkdir -p objs/
g++ -m64 -O3 -Wall -o cudaSaxpy objs/main.o  objs/saxpy.o -L/usr/local/cuda/lib64/ -lcudart
---------------------------------------------------------
Found 1 CUDA devices
Device 0: Tesla T4
   SMs:        40
   Global mem: 15102 MB
   CUDA Cap:   7.5
---------------------------------------------------------
Running 3 timing tests:
Effective BW by CUDA saxpy: 4.704 ms		[237.590 GB/s]
Effective BW by CUDA saxpy: 4.631 ms		[241.325 GB/s]
Effective BW by CUDA saxpy: 4.634 ms		[241.173 GB/s]


## 2. Parallel Prefix-Sum 구현

In [7]:
# saxpy 폴더에서 scan 폴더로 이동하기 위해 상위 폴더로 이동합니다.

%cd ..

/content/drive/MyDrive/대학원 관련/c언어/asst3-master/asst3-master


In [8]:
!dir

cloud_readme.md		   handout     README.md  saxpy
CUDA\ asst3.ipynb의\ 사본  install.sh  render	  scan


In [11]:
%cd scan/

/content/drive/MyDrive/대학원 관련/c언어/asst3-master/asst3-master/scan


In [12]:
!dir

checker.pl  cudaScan_ref  main.cpp  objs
cudaScan    CycleTimer.h  Makefile  scan.cu


In [13]:
!make

mkdir -p objs/
g++ -m64 -O3 -Wall -o cudaScan objs/main.o  objs/scan.o -L/usr/local/cuda/lib64/ -lcudart


In [ ]:
!dir

checker.pl  cudaScan_ref  main.cpp  objs
cudaScan    CycleTimer.h  Makefile  scan.cu


In [14]:
!./cudaScan

---------------------------------------------------------
Found 1 CUDA devices
Device 0: Tesla T4
   SMs:        40
   Global mem: 15102 MB
   CUDA Cap:   7.5
---------------------------------------------------------
Array size: 64
Student GPU time: 0.250 ms
Scan outputs are correct!
